 Read in the data and create objects to pass onto Gurobi constraints. Second column in parcel file is area and third column in parcel file is price

In [1]:
import numpy as np
import pandas as pd
parcels_data = np.array(pd.read_csv("Parcels.txt", header = None))

In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
P = parcels_data[:,0]
P = np.array(P.astype(int))
print(P)
type(P)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120]


numpy.ndarray

In [4]:
A = parcels_data[:,round(1, 5)]
C = parcels_data[:,round(2, 8)]

In [5]:
multidict_input = {}
for p in P:
    multidict_input[p] = [A[p-1], C[p-1]]
#print(multidict_input)

In [6]:
adjacency_data = pd.read_csv("Adjacency.txt", header = None)
print(adjacency_data)
subset = adjacency_data[[0,1]]
adjacency = [tuple(x) for x in subset.values]
print(adjacency)
type(adjacency)

       0   1
0      2  51
1      3  50
2      3  55
3      3  60
4      4   5
..   ...  ..
255   98  96
256   98  99
257   99  97
258   99  98
259  100  90

[260 rows x 2 columns]
[(2, 51), (3, 50), (3, 55), (3, 60), (4, 5), (4, 6), (4, 48), (5, 4), (5, 54), (6, 4), (6, 47), (6, 56), (7, 57), (7, 58), (8, 60), (8, 66), (8, 68), (10, 13), (12, 63), (12, 73), (13, 10), (13, 16), (14, 17), (14, 19), (14, 21), (14, 22), (14, 24), (15, 17), (15, 18), (15, 23), (15, 75), (16, 13), (16, 20), (16, 76), (17, 14), (17, 15), (17, 19), (17, 23), (17, 64), (18, 15), (18, 73), (18, 75), (19, 14), (19, 17), (19, 22), (19, 23), (20, 16), (20, 76), (21, 14), (22, 14), (22, 19), (22, 23), (22, 24), (23, 15), (23, 17), (23, 19), (23, 22), (23, 26), (23, 27), (23, 75), (24, 14), (24, 22), (25, 77), (25, 78), (26, 23), (26, 27), (27, 23), (27, 26), (27, 28), (27, 32), (27, 75), (28, 27), (28, 31), (28, 32), (28, 33), (29, 31), (29, 76), (31, 28), (31, 29), (31, 33), (31, 35), (31, 37), (31, 76), (32, 27), 

list

In [7]:
# what gurobi py uses when formulating constraints/objective function
parcels, areas, costs = gp.multidict(multidict_input)
print(parcels)
# this is where we would specify our budget
budget = 1000000

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]


Now that the data is in and the budget is specified, I can write out the model

In [8]:
# create model object
m = gp.Model('ParcelSelct')

Restricted license - for non-production use only - expires 2022-01-13


In [9]:
# create decision variables for each of the parcels and flow variables
x = m.addVars(parcels, vtype = GRB.BINARY, name = "parcel")
y = m.addVars(adjacency, vtype = GRB.BINARY, name = "flow")


In [10]:
# create budget constraint
budget = m.addConstr((x.prod(costs) <= budget), name = 'budget')

# create core parcel constraint
core = m.addConstr((x[23] == 1), name = 'corep')


In [12]:
# Constraint 3 from paper
for p in parcels:
    A = [a[0] for a in adjacency  
            if a[1]==p]
    if not A:  
        pass
    else:
        m.addConstr(sum(y[i,p] for i in A) <= len(A)*x[p], name = "no_flow" + str(p))

In [14]:
# Constraint 4 from paper
for p in parcels:
    A = [a[1] for a in adjacency  #finds all parcels adjacent to parcel i
            if a[0]==p]
    if not A:
        pass  #skips this constraint if there are no parcels adjacent to i
    else:
        m.addConstr(sum(y[p,j] for j in A) <= x[p], name = "one_flow_to" + str(p)) 

In [15]:
# Constraint 5 from paper
connected = m.addConstr(y.sum()== x.sum() - 1, name = "arcs_to_nodes")

In [16]:
z = m.addVars(adjacency, vtype = GRB.INTEGER, name = "Z") #tail length contribution variable
w = m.addVars(parcels, vtype = GRB.INTEGER, name = "W") #tail length variable

In [17]:
M = len(parcels) + 1 

In [18]:
for p in parcels:
    A = [a[1] for a in adjacency #finds all parcels adjacent to i
            if a[0]==p]
    if not A: #if there are no parcels adjacent to i, skips this constraint
        pass
    else:
         for j in A:
            if j != p: #for each j where j does not equal i, calculates tail contribution
                m.addConstr(z[p,j] >= w[p] + 1 - M*(1 - y[p,j]), name = "tail_length" + str(p)+str(j))

In [19]:
#tail length W
for p in parcels:
    A = [a[0] for a in adjacency #finds all parcels adjacent to j
            if a[1] == p]
    if not A:
        pass #skips this constraint if there are no parcels adjacent to j
    else:
        m.addConstr(w[p] == z.sum('*',p), name = "wtail" + str(p)) #calculates w[j]

In [20]:
# Create objective function

m.setObjective(x.prod(areas), GRB.MAXIMIZE)

In [21]:
# then we can write out the model to upload later
m.write('ParcelSelect.lp') # this seems to be the basic model format, 